# 连接仪器

In [2]:
from qulab.sugar import connect
from qulab.job import Job
from qulab.storage.utils import save
import matplotlib.pyplot as plt, os
import numpy as np, sympy as sy, visa
from collections import namedtuple, deque, defaultdict, OrderedDict, Counter
from scipy.optimize import curve_fit, least_squares as ls
import time, asyncio, sys, imp, pandas as pd
from tqdm import tqdm_notebook as tqdm
from easydl import clear_output
from sklearn.cluster import KMeans
from qulab import wavepoint 
from qulab.wavepoint import WAVE_FORM as WF
from qulab import optimize, measureroutine, computewave, imatrix, waveform_new
# from qulab.optimize import Collect_Waveform 
from qulab.waveform import CosPulseDrag, Expi, DC, Step, Gaussian
from qulab.math.qst import transformList, acquireVFromData, vToRho
from qulab.math import opt
opt = imp.reload(opt)
op = imp.reload(optimize)
mr = imp.reload(measureroutine)
cw = imp.reload(computewave)
mx = imp.reload(imatrix)
wn = imp.reload(waveform_new)

In [22]:
measure = mr.common(0,0,0,0,0,0)

In [3]:
rm = visa.ResourceManager()
spec = rm.open_resource('TCPIP::10.122.7.191')
awg = await connect('AWG133')
await awg.setValue('Run Mode','Continuous')    # Triggered
f_cent = 4.238e9
spec.write('FREQ:CENT %f'%f_cent)

(29, <StatusCode.success: 0>)

# 衰减器归零

In [24]:
# for i in measure.com_lst:
#     mr.Att_Setup(measure,i).Att(0)

# 工作点

In [4]:
atributte = ['q_name','alpha_read','alpha_ch1','alpha_ch2','alpha_ch3','alpha_ch4','alpha_ch5',\
            'beta_read','beta_ch1','beta_ch2','beta_ch3','beta_ch4','beta_ch5']
Iqmixer = namedtuple('Iqmixer',atributte,defaults=[{} for i in range(12)])

iqm = Iqmixer(q_name='Iqm') #power=-2.1

# 保存IQ-Mixer状态

In [140]:
qstate = []
def tuple2dict(q):
    for i in q:
        qstate.append(i._asdict())
    return qstate

filepath = save('IQMixerState','iqmixerstate',state = tuple2dict([iqm]),base_path=r'D:\skzhao\file_name')
with open(r'D:\skzhao\file_name\iqmixer_state.txt', mode='a') as filename:
    filename.write(str(filepath))
    filename.write('\n')
print(filepath)

D:\skzhao\file_name\2020\0708\IQMixerState\iqmixerstate_20200708130458.npz


# 提取IQ-Mixer状态

In [5]:
fl = np.loadtxt(r'D:\skzhao\file_name\iqmixer_state.txt',dtype='str')
f = str(fl[-1])
data = np.load(f,allow_pickle=True)
def dict2tuple(q):
    global iqm
    for i,k in enumerate(q):
        attr = []
        for j in atributte:
            if j in k:
                attr.append(k[j])
            else:
                attr.append(None)
        iqm = Iqmixer(*attr)

dict2tuple(data['state'])
print(f)

D:\skzhao\file_name\2020\0708\IQMixerState\iqmixerstate_20200708130458.npz


In [6]:
iqm

Iqmixer(q_name='Iqm', alpha_read={}, alpha_ch1={}, alpha_ch2={'3.700000': array([ 9.000e-03, -3.500e-02, -5.096e+01]), '3.710000': array([ 5.000e-03, -3.700e-02, -4.909e+01]), '3.720000': array([ 3.00e-03, -3.90e-02, -5.27e+01]), '3.730000': array([ 1.000e-03, -4.000e-02, -5.184e+01]), '3.740000': array([-1.000e-03, -4.000e-02, -4.839e+01]), '3.750000': array([-2.000e-03, -4.100e-02, -4.951e+01]), '3.760000': array([-4.000e-03, -4.100e-02, -5.851e+01]), '3.770000': array([-5.000e-03, -4.100e-02, -5.005e+01]), '3.780000': array([-6.000e-03, -4.000e-02, -4.755e+01]), '3.790000': array([-7.000e-03, -4.000e-02, -4.957e+01]), '3.800000': array([-8.000e-03, -4.000e-02, -4.702e+01]), '3.810000': array([-1.000e-02, -3.900e-02, -4.516e+01]), '3.820000': array([-1.000e-02, -3.900e-02, -5.323e+01]), '3.830000': array([-1.100e-02, -3.900e-02, -4.999e+01]), '3.840000': array([-1.300e-02, -3.900e-02, -5.137e+01]), '3.850000': array([-1.400e-02, -3.900e-02, -4.885e+01]), '3.860000': array([-1.500e-02

# 找频率位置

In [5]:
f_center = eval(spec.query('FREQuency:CENTer?').strip('\n'))
f_start = spec.query_ascii_values(':FREQ:START?')[0]
f_stop = spec.query_ascii_values(':FREQ:STOP?')[0]
y = np.asarray(spec.query_ascii_values(':TRACe? TRACE1 '))
f = np.linspace(f_start, f_stop, len(y))
np.abs(f-4.1318e9).argmin()

500

# 压制本振

In [7]:
ch_I, ch_Q = 2,6
await cw.genwaveform(awg,['opt_I','opt_Q'],[ch_I,ch_Q])
pulse = await cw.rabiWave(envelopename='square',during=0/1e9,shift=0,DRAGScaling=None)
await cw.writeWave(awg,['opt_I','opt_Q'],pulse)
await awg.output_on(ch=ch_I)
await awg.output_on(ch=ch_Q)
await cw.couldRun(awg)

In [8]:
async def SNR(offset1,offset2):
    global ch_I, ch_Q
    await awg.setValue('Offset',offset1,ch=ch_I)
    await awg.setValue('Offset',offset2,ch=ch_Q)
    time.sleep(2)
    await cw.couldRun(awg)
    y = []
    for i in range(20):
        time.sleep(0.5)
        y.append(np.asarray(spec.query_ascii_values(':TRACe? TRACE1 ')))
    y = np.mean(y,axis=0)
#     print(np.mean(y[480:520]))
    return np.mean(y[490:510])

In [12]:
start = [-0.05,0.05]
senstive = np.abs(start)/10
ret = opt.optimize(SNR,start,senstive,[3,3],high=[2,2],low=[-2,-2],print_info=True,algorithm='minimize')

.o
[-0.05  0.05] -37.328128874825   .o
[-0.045  0.05 ] -37.5448875012   ..o
[-0.045  0.045] -37.82490399975   .o
[-0.043  0.04 ] -38.2148910031   .o
[-0.038  0.04 ] -38.591645419125   .o
[-0.031  0.035] -39.567034339799996   .o
[-0.029  0.025] -40.91273286017499   .o
[-0.021  0.013] -43.84895802639999   .o
[-0.009  0.008] -46.857801367549996   .o
[ 0.007 -0.009] -53.899270425175   .......o
[ 0.006 -0.012] -55.74745419324999   ...o
[-0.    -0.011] -60.1995336582   .o
[-0.004 -0.012] -62.197265023125   .o
[ 0.    -0.017] -64.545086548675   ...o
[-0.006 -0.016] -65.33810413885   ..o
[-0.003 -0.014] -67.776248548825   ..o
[-0.002 -0.016] -72.46310522355   ..o
[-0.004 -0.016] -72.92076632092501   .o
[-0.003 -0.017] -74.49108346202499   ...o
[-0.003 -0.016] -76.78543580450003   ................
 CacheInfo(hits=14, misses=38, maxsize=128, currsize=38)


In [13]:
ret.x

array([-0.003, -0.016])

In [15]:
await awg.setValue('Offset',-0.003,ch=ch_I)
await awg.setValue('Offset',-0.016,ch=ch_Q)

In [33]:
del iqm.beta_ch2['4106716000.000000']

In [47]:
iqm.alpha_ch4['%f'%(f_cent/1e9)] = [-0.076, -0.225]

# 压制镜像

In [134]:
rm = visa.ResourceManager()
spec = rm.open_resource('TCPIP::10.122.7.191')
awg = await connect('AWG132')
await awg.setValue('Run Mode','Continuous')    # Triggered
psg_ex1 = await connect('PSG105')

In [135]:
await awg.clear_waveform_list()
for j in range(8):
    await awg.output_off(ch=j+1)

In [136]:
ch_I, ch_Q = 1,2
t_new = t_new = np.linspace(-50000,50000,250000)*1e-9
iqm_ch = iqm.alpha_ch4

await cw.genwaveform(awg,['opt_I','opt_Q'],[ch_I,ch_Q])
pulse = await cw.rabiWave(envelopename='square',during=0/1e9,shift=0,DRAGScaling=None)
await cw.writeWave(awg,['opt_I','opt_Q'],pulse)
await awg.output_on(ch=ch_I)
await awg.output_on(ch=ch_Q)
await cw.couldRun(awg)

In [137]:
async def ImReject_wave(phasediff,amp1=1,amp2=1,during=110e-6,Delta_lo=200e6):
    global ch_I, ch_Q, awg, t_new
    pulse = wn.square(during)
    wav_I, wav_Q = wn.mixing(pulse,phase=0,freq=Delta_lo,ratioIQ=-1.0,phaseDiff=phasediff)
    await cw.writeWave(awg,['opt_I','opt_Q'],(amp1*wav_I,amp2*wav_Q,wn.zero(),wn.zero()),t=t_new)
    await cw.couldRun(awg)

In [138]:
async def phaseSNR(phasediff,amp1,amp2):
    await ImReject_wave(phasediff,amp1,amp2)
    time.sleep(2)
    await cw.couldRun(awg)
    y = []
    for i in range(30):
        time.sleep(0.2)
        y.append(np.asarray(spec.query_ascii_values(':TRACe? TRACE1 ')))
    y = np.mean(y,axis=0)
#     print(np.mean(y[480:520]))
    return np.mean(y[490:510])

In [73]:
i = 4e9
await psg_ex1.setValue('Output','ON')
await psg_ex1.setValue('Frequency',i)
para = iqm_ch['%.6f'%(i/1e9)]
await awg.setValue('Offset',para[0],ch=ch_I)
await awg.setValue('Offset',para[1],ch=ch_Q)
spec.write('FREQ:CENT %f'%(i+200e6))
print(round(i/1e9,4))
opt.optimize(phaseSNR,[0,1,1],[np.pi/20,0.01,0.01],[5,3,3],high=[np.pi,1,1],low=[-np.pi,0,0],print_info=True)

4.0
.o
[0. 1. 1.] -32.249285856916664   .o
[0.15708 1.      1.     ] -37.563752245316664   ............o
[0.17063 0.992   1.     ] -37.9686152808   ........o
[0.16763 0.991   1.     ] -38.04266755233333   .o
[0.1729 0.986  1.    ] -38.5309897478   ...o
[0.17734 0.976   1.     ] -39.19967553711666   .o
[0.1807 0.968  1.    ] -39.724566011166665   ..o
[0.17816 0.97    1.     ] -39.75621110385   ..o
[0.17046 0.961   1.     ] -40.99486994866666   .o
[0.16925 0.948   1.     ] -42.40430503353333   ....o
[0.17523 0.927   1.     ] -42.96017236611667   ..o
[0.16027 0.928   1.     ] -45.922624123666665   .o
[0.14743 0.916   1.     ] -50.40167059141667   ....o
[0.13489 0.922   1.     ] -55.3965154576   ..........o
[0.13526 0.914   1.     ] -58.52207622708333   ....o
[0.12917 0.915   1.     ] -61.2554357441   ....o
[0.13105 0.916   1.     ] -65.69378754393333   ...........o
[0.13199 0.916   1.     ] -66.70229221995001   ..........o
[0.13176 0.916   1.     ] -66.74479603581668   .......o
[0.13199 0

 final_simplex: (array([[0.13198853, 0.91544729, 1.08793714],
       [0.13198906, 0.91544837, 1.08793804],
       [0.13199012, 0.91545215, 1.0879319 ],
       [0.1319949 , 0.91544924, 1.08793536]]), array([-66.82632647, -66.82632647, -66.82632647, -66.82632647]))
           fun: -66.82632647173335
       message: 'Optimization terminated successfully.'
          nfev: 122
           nit: 60
        status: 0
       success: True
             x: array([0.13199, 0.915  , 1.     ])

In [118]:
await ImReject_wave(0)

In [139]:
# start = [-0.05,0.05]
# senstive = np.abs(start)/10
await psg_ex1.setValue('Output','ON')
for j, i in enumerate(np.arange(3.7,4.3,0.01)*1e9,start=1):
    os.environ['CUDA_VISIBLE_DEVICES'] = '1'
    await psg_ex1.setValue('Frequency',i)
    para = iqm_ch['%.6f'%(i/1e9)]
    await awg.setValue('Offset',para[0],ch=ch_I)
    await awg.setValue('Offset',para[1],ch=ch_Q)
    spec.write('FREQ:CENT %f'%(i+200e6))
    print(round(i/1e9,4))
    try:
        ret = opt.optimize(phaseSNR,[0,1,1],[np.pi/20,0.01,0.01],[5,3,3],high=[np.pi,1,1],low=[-np.pi,0,0],print_info=True)
        coef = ret.x
        coef = np.append(coef,round(ret.fun,2))
        iqm_ch['%.6f'%(i/1e9)] = np.append(iqm_ch['%.6f'%(i/1e9)],coef)
    except:
        pass
    if j % 5 == 0:
        clear_output()
        filepath = save('IQMixerState','iqmixerstate',state = tuple2dict([iqm]),base_path=r'D:\skzhao\file_name')
        with open(r'D:\skzhao\file_name\iqmixer_state.txt', mode='a') as filename:
            filename.write(str(filepath))
            filename.write('\n')

In [115]:
spec.write('FREQ:CENT %f'%4.02e9)

(29, <StatusCode.success: 0>)

In [117]:
await psg_ex1.setValue('Frequency',4.02e9)

In [133]:
len(np.arange(3.7,4.3,0.01)*1e9)

60

In [126]:
iqm_ch

{'3.700000': array([ 1.400e-02, -3.000e-02, -6.256e+01]),
 '3.710000': array([ 7.000e-03, -3.000e-02, -6.183e+01]),
 '3.720000': array([ 3.000e-03, -3.000e-02, -6.284e+01]),
 '3.730000': array([-0.000e+00, -2.900e-02, -6.015e+01]),
 '3.740000': array([-3.000e-03, -2.900e-02, -6.057e+01]),
 '3.750000': array([-4.000e-03, -2.600e-02, -6.189e+01]),
 '3.760000': array([-1.000e-02, -3.100e-02, -6.251e+01]),
 '3.770000': array([-1.00e-02, -3.50e-02, -6.14e+01]),
 '3.780000': array([-1.000e-02, -3.400e-02, -6.054e+01]),
 '3.790000': array([-1.100e-02, -3.300e-02, -5.834e+01]),
 '3.800000': array([-1.100e-02, -3.300e-02, -6.408e+01]),
 '3.810000': array([-1.200e-02, -3.400e-02, -6.244e+01]),
 '3.820000': array([-1.400e-02, -3.500e-02, -6.182e+01]),
 '3.830000': array([-1.500e-02, -3.500e-02, -5.919e+01]),
 '3.840000': array([-1.50e-02, -3.30e-02, -6.09e+01]),
 '3.850000': array([-1.500e-02, -3.300e-02, -6.005e+01]),
 '3.860000': array([-1.600e-02, -3.300e-02, -6.162e+01]),
 '3.870000': array([

# 连续压制本振

In [81]:
rm = visa.ResourceManager()
spec = rm.open_resource('TCPIP::10.122.7.191')
awg = await connect('AWG132')
await awg.setValue('Run Mode','Continuous')    # Triggered
psg_ex1 = await connect('PSG105')

In [82]:
ch_I, ch_Q = 5,6
iqm_ch = iqm.alpha_ch5

await cw.genwaveform(awg,['opt_I','opt_Q'],[ch_I,ch_Q])
pulse = await cw.rabiWave(envelopename='square',during=0/1e9,shift=0,DRAGScaling=None)
await cw.writeWave(awg,['opt_I','opt_Q'],pulse)
await awg.output_on(ch=ch_I)
await awg.output_on(ch=ch_Q)
await cw.couldRun(awg)

In [83]:
async def SNR(offset1,offset2):
    global ch_I, ch_Q
    await awg.setValue('Offset',offset1,ch=ch_I)
    await awg.setValue('Offset',offset2,ch=ch_Q)
    time.sleep(2)
    await cw.couldRun(awg)
    y = []
    for i in range(30):
        time.sleep(0.2)
        y.append(np.asarray(spec.query_ascii_values(':TRACe? TRACE1 ')))
    y = np.mean(y,axis=0)
#     print(np.mean(y[480:520]))
    return np.mean(y[490:510])

In [84]:
start = [-0.08,0.08]
senstive = np.abs(start)/10
await psg_ex1.setValue('Output','ON')
for j, i in enumerate(np.array([4])*1e9,start=1):
    os.environ['CUDA_VISIBLE_DEVICES'] = '1'
    await psg_ex1.setValue('Frequency',i)
    spec.write('FREQ:CENT %f'%i)
    print(round(i/1e9,4))
    try:
        ret = opt.optimize(SNR,start,senstive,[3,3],high=[2,2],low=[-2,-2],print_info=True,algorithm='minimize')
        coef = ret.x
        coef = np.append(coef,round(ret.fun,2))
        iqm_ch['%f'%(round(i/1e9,4))] = coef
    except:
        pass
    if j % 5 == 0:
        clear_output()
        filepath = save('IQMixerState','iqmixerstate',state = tuple2dict([iqm]),base_path=r'D:\skzhao\file_name')
        with open(r'D:\skzhao\file_name\iqmixer_state.txt', mode='a') as filename:
            filename.write(str(filepath))
            filename.write('\n')

4.0
.o
[-0.08  0.08] -7.854332601690001   .o
[-0.072  0.08 ] -7.9766164739   ..o
[-0.072  0.072] -8.206500685721668   .o
[-0.068  0.064] -8.608885134781666   .o
[-0.06   0.064] -8.7872900251   .o
[-0.05   0.056] -9.417634057920665   .o
[-0.046  0.04 ] -10.668769956756584   .o
[-0.033  0.02 ] -13.108763464842166   .o
[-0.015  0.012] -14.308475717541835   .o
[ 0.011 -0.014] -15.155060956086334   ..o
[ 0.009 -0.023] -16.18367357140017   ...o
[-0.014 -0.009] -18.998907239338333   ...o
[-0.017 -0.018] -22.847785780248334   .....o
[-0.019 -0.028] -28.938373271416662   .o
[-0.027 -0.03 ] -37.622151935533324   .........o
[-0.028 -0.033] -40.334346652983335   ..o
[-0.026 -0.034] -41.59714157321666   ..o
[-0.027 -0.032] -45.27953076791667   ..o
[-0.027 -0.033] -45.49464617791667   ..o
[-0.026 -0.033] -49.268213007133326   ....o
[-0.026 -0.032] -50.6551227715   ...............
 CacheInfo(hits=17, misses=44, maxsize=128, currsize=44)


In [15]:
# spec.write('FREQ:CENT %f'%4.08e9)

In [11]:
len(np.arange(3.7,4.3,0.01)*1e9)

60

In [66]:
ret.x

array([-0.031, -0.046])

In [79]:
iqm.alpha_ch5

{'3.700000': array([-2.800e-02,  3.000e-03, -6.416e+01]),
 '3.720000': array([-3.100e-02, -8.000e-03, -6.496e+01]),
 '3.740000': array([-3.100e-02, -1.200e-02, -6.293e+01]),
 '3.760000': array([-3.200e-02, -1.800e-02, -6.132e+01]),
 '3.780000': array([-3.400e-02, -1.900e-02, -6.271e+01]),
 '3.800000': array([-3.400e-02, -2.000e-02, -6.302e+01]),
 '3.820000': array([-3.600e-02, -2.300e-02, -6.032e+01]),
 '3.840000': array([-3.600e-02, -2.300e-02, -6.484e+01]),
 '3.860000': array([-3.400e-02, -2.300e-02, -6.224e+01]),
 '3.880000': array([-3.300e-02, -2.400e-02, -6.339e+01]),
 '3.900000': array([-3.200e-02, -2.500e-02, -6.438e+01]),
 '3.920000': array([-3.100e-02, -2.700e-02, -6.504e+01]),
 '3.940000': array([-3.000e-02, -2.800e-02, -6.562e+01]),
 '3.960000': array([-3.10e-02, -3.00e-02, -6.33e+01]),
 '3.980000': array([-3.000e-02, -3.100e-02, -6.375e+01]),
 '4.000000': array([-2.200e-02,  1.200e-02, -6.597e+01]),
 '4.020000': array([-3.300e-02, -3.600e-02, -6.384e+01]),
 '4.040000': arra

In [17]:
er = [4.16]

In [13]:
len(er)

24

In [48]:
iqm_ch['4.000000']

array([-3.100e-02, -4.500e-02, -5.133e+01])

In [51]:
float(4.000000)

4.0

In [85]:
await psg_ex1.setValue('Output','OFF')